Implementations of simple greedy vs brute force algorithms; used to demonstrate real run time

In [58]:
#Greedy algorithm
#implemented using an edge_list

def addEdge(edge_list, vertex_1, vertex_2):
    edge_list[vertex_1].append(vertex_2) #adds vertex_2 to vertex_1s adjacencies
    edge_list[vertex_2].append(vertex_1) #adds vertex_1 to vertex_2s adjacencies

    return edge_list #returns the entire edge list

def greedy(edge_list):
    colors = [-1] * len(edge_list) #create an array of length num_vertices filled with -1s; this array will hold the colors for each vertex
    available_colors = [False] * len(edge_list) #create an array of length num_vertices filled with False; this array will hold which colors are available for that vertex


    colors[0] = 0 #assign the first vertex to the first color

    for vertex in range(1, len(edge_list)): #loop through all of the vertices except the first one, since that one is already color 0
        for adjacent_vertex in edge_list[vertex]: #loop through all of the adjacencies for that vertex
            if colors[adjacent_vertex] != -1: #if the color is not taken by any of the adjacent vertices
                available_colors[colors[adjacent_vertex]] = True #then it is an available color for this vertex
        
        lowest_color = 0
        
        while lowest_color < len(edge_list):
            if available_colors[lowest_color] == False:
                break
            lowest_color += 1
        
        colors[vertex] = lowest_color
        available_colors = [False] * len(edge_list) # reset available colors for next loop
    
    for vertex in range(len(edge_list)):
        print(f"Vertex {vertex} is adjacent to vertices {edge_list[vertex]}")

    for vertex in range(len(edge_list)):
        print(f"Vertex {vertex} is colored {colors[vertex]}") #print all of the vertices and their color

In [69]:
import itertools
import math
#Brute Force Algorithm
#Implemented using an adjacency_matrix
def add_edge(adjacency_matrix, vertex_1, vertex_2):
    adjacency_matrix[vertex_1][vertex_2] == 1
    adjacency_matrix[vertex_2][vertex_1] == 1

def isValidColoring(adjacency_matrix, vertex_colors):
    for i in range(int(math.sqrt(len(adjacency_matrix)))): #loop through each vertex
        for j in range(int(math.sqrt(len(adjacency_matrix)))): #loop through all of the other vertices that aren't our current vertex
            if adjacency_matrix[i][j] == 1 and vertex_colors[i] == vertex_colors[j]: #if they are adjacent the same color
                return False #this is not a valid coloring
    return True #if you loop through everything and never run into a problem, it is a valid coloring

def bruteForce(adjacency_matrix, num_colors):
    all_vertex_colors = allPossibleColorings(adjacency_matrix, num_colors)

    for i in range(int(math.sqrt(len(adjacency_matrix)))):
        for j in range(int(math.sqrt(len(adjacency_matrix)))):
            if adjacency_matrix[i][j] == 1:
                print(f"Vertex {i} is adjacent to vertex {j}")
    
    for possible_coloring in all_vertex_colors:
        if isValidColoring(adjacency_matrix, possible_coloring) == True:
            print("Valid Coloring Found!")
            for vertex in range(int(math.sqrt(len(adjacency_matrix)))):
                print(f"Vertex {vertex} is colored {possible_coloring[vertex]}")
            return
    
    print("No Valid Coloring with {num_colors} colors")


def allPossibleColorings(adjacency_matrix, num_colors):
    all_vertex_colors = []
    colors = [i for i in range(num_colors)]

    for p in itertools.product(colors, repeat=int(math.sqrt(len(adjacency_matrix)))):
        all_vertex_colors.append(p)
    
    return all_vertex_colors




In [72]:
if __name__ == '__main__':
     
    # g1 = [[] for _ in range(5)] #create empty edge_list for five vertices
    # g1 = addEdge(g1, 0, 1)
    # g1 = addEdge(g1, 0, 2)
    # g1 = addEdge(g1, 1, 2)
    # g1 = addEdge(g1, 1, 3)
    # g1 = addEdge(g1, 2, 3)
    # g1 = addEdge(g1, 3, 4)
    # greedy(g1)

    g2 = [[0] for _ in range(3)] * 3 #create an empty adjacency_matrix for three vertices
    g2 = addEdge(g2, 0, 1)
    g2 = addEdge(g2, 0, 2)
    g2 = addEdge(g2, 1, 2)
    bruteForce(g2, 3)
    
        

Vertex 0 is adjacent to vertex 1
Vertex 2 is adjacent to vertex 2
No Valid Coloring with {num_colors} colors
